In [1]:
import os
import pandas as pd
import sys
import shutil
import json

sys.path.append("../")
from utils.color_detection import color_detection, FabricPattern

## Load DB tables

In [2]:
# Annotated mask
path_db = "../db"
df_tiles = pd.read_csv(os.path.join(path_db,"tiles.csv"))
df_defects = pd.read_csv(os.path.join(path_db,"defects.csv"))
df_rolls = pd.read_csv(os.path.join(path_db, "rolls.csv"))

In [3]:
# Select roll by color or pattern
df_rolls = df_rolls[(df_rolls["DispositionNumber"] > 0) & (df_rolls["DispositionRowNumber"] > 0)]
df_rolls.head(2)

,Unnamed: 0,Id,MachineId,RollCounter,DispositionNumber,DispositionRowNumber,RollNumber,CreationTime,CustomerArticle,CustomerDesign,CustomerVariant,SupplierArticle,SupplierDesign,SupplierVariant,Composition,NumTileRows,Color,Type
5,5,34,1,32,966482,1,13408011,2023-11-14 15:42:26.038780+00:00,99TPC02-PC02,B0310,NaN,LINONE 06,6199 3-12 2,NaN,LC67 67% Lino 33% Cotone,281,['black'],uniform
6,6,7,1,5,966201,1,13404383,2023-11-13 15:01:14.589832+00:00,10258454,100,NaN,NaN,NaN,NaN,100 CO,30,[],NaN


In [4]:
df_tiles.head(2)

,Unnamed: 0,Id,RollId,Row,Column,AbsVisImageFileName,AbsIrImageFileName,CreationTime,Tags
0,0,2,2,15,2,cqt_qualitex_fino/specola_20/M0/M0_R15_C2_VIS.jpg,cqt_qualitex_fino/specola_20/M0/M0_R15_C2_IR.jpg,2023-10-26 14:07:24.856315+00:00,[]
1,1,3,2,14,2,cqt_qualitex_fino/specola_20/M0/M0_R14_C2_VIS.jpg,cqt_qualitex_fino/specola_20/M0/M0_R14_C2_IR.jpg,2023-10-26 14:07:24.226122+00:00,[]


In [5]:
df_defects.head(2)

,Unnamed: 0,Id,TileId,Uuid,ShapePointsX,ShapePointsY,Type,CategoryAi,Score,ScoreThreshold,State,Tags,TilePreprocessingFilter,Channel,CreatedBy,ValidatedByOperator,ModifiedByOperator,CreationTime
0,0,2,2,7eb84c3f-7409-11ee-9a5b-00be438fbbe2,"[0.3857187045280588, 0.38655413143533535, 0.38...","[1.0, 0.9966582923708964, 0.9933505271424874, ...",67.0,NaN,0.0,0.0,permanent,['certain'],clahe,VIS,operator,False,False,2023-10-26 14:11:21.115130+00:00
1,1,3,3,7eb84c3f-7409-11ee-9a5b-00be438fbbe2,"[0.36049118878027686, 0.36086996076086164, 0.3...","[0.0, 0.00161340476587446, 0.01276240765457492...",67.0,NaN,0.0,0.0,permanent,['certain'],clahe,VIS,operator,False,False,2023-10-26 14:11:21.136291+00:00


## Filter tiles per color pattern

In [7]:
start_search = 100
stop_search = 200

In [8]:
roll_ids = df_rolls.Id.values
path_download = "./color_test/"
black_fabrics = "./color_test/invalid/"
os.makedirs(black_fabrics, exist_ok=True)
s3_bucket = "aqc-lambda-tiles-prod"
pallette_path = os.path.dirname(os.path.realpath("../utils/pallettes/pallette.json"))
filter_rolls = []

# limit the number of rolls to check
# Rolls:
# 0-100: done!
for roll_id in roll_ids[start_search:stop_search]:
    df_tile_color = df_tiles[(df_tiles["RollId"] == roll_id) & 
                            (df_tiles["AbsVisImageFileName"].str.contains("C1")) &
                            (df_tiles["Tags"].str.contains("training"))].reset_index(drop=True)
    position = int(len(df_tile_color)/2)
    if position > 0:
        disposition_n = df_rolls[df_rolls["Id"] == roll_id]["DispositionNumber"].values[0]
        disposition_r = df_rolls[df_rolls["Id"] == roll_id]["DispositionRowNumber"].values[0]
        disposition = "_".join([str(disposition_n), str(disposition_r)])
        # Download image to test color detection
        object_name = df_tile_color["AbsVisImageFileName"].values[position]
        file_name = object_name.split("/")[-1]
        machine_roll = object_name.split("/")[-2]
        # Only download one image per disposition
        os.makedirs(os.path.join(path_download, disposition, machine_roll), exist_ok=True)
        if len(os.listdir(os.path.join(path_download, disposition))) < 2:
            path_to_save = os.path.join(path_download, disposition, machine_roll, file_name)
        
            if not os.path.exists(path_to_save):
                !aws s3 cp s3://$s3_bucket/$object_name $path_to_save
        
            # Color detection
            color, value, pattern = color_detection([path_to_save], pallette=pallette_path)
            if "black" in color and FabricPattern(pattern).value == 1:
                if not os.path.exists(os.path.join(black_fabrics, disposition)):
                    shutil.move(os.path.join(path_download, disposition), black_fabrics)
                else:
                    shutil.rmtree(os.path.join(path_download, disposition))
            else:
                filter_rolls.append(roll_id)
        else:
            shutil.rmtree(os.path.join(path_download, disposition, machine_roll))

download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M104/M104_R60_C1_VIS.jpg to color_test/967235_1/M104/M104_R60_C1_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M131/M131_R8_C1_VIS.jpg to color_test/968141_1/M131/M131_R8_C1_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M132/M132_R65_C1_VIS.jpg to color_test/968141_1/M132/M132_R65_C1_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M133/M133_R78_C1_VIS.jpg to color_test/968141_1/M133/M133_R78_C1_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M125/M125_R62_C1_VIS.jpg to color_test/968141_1/M125/M125_R62_C1_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M134/M134_R456_C1_VIS.jpg to color_test/968141_1/M134/M134_R456_C1_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M163/M163_R74_C1_VIS.jpg to color_test/968129_1/M163/M163_R74_C1_VIS.jpg
download: s3://aqc-lambda-t

In [9]:
filter_rolls

[165, 169, 173, 472, 196]

## Download defective tiles

In [10]:
for roll_id in filter_rolls:
    disposition_n = df_rolls[df_rolls["Id"] == roll_id]["DispositionNumber"].values[0]
    disposition_r = df_rolls[df_rolls["Id"] == roll_id]["DispositionRowNumber"].values[0]
    disposition = "_".join([str(disposition_n), str(disposition_r)])
    rolls = df_rolls[(df_rolls["DispositionNumber"] == disposition_n) &
                    (df_rolls["DispositionRowNumber"] == disposition_r)]
    tiles = df_tiles[(df_tiles["RollId"].isin(rolls.Id.values)) &
                    (df_tiles["Tags"].str.contains("\[]"))]
    defects = df_defects[(df_defects["TileId"].isin(tiles.Id.values)) &
                        (df_defects["CreatedBy"] == "operator")]
    tiles = tiles[tiles["Id"].isin(defects.TileId.values)]

    # Download defective images
    path_download = os.path.join(f"../annotations/{disposition}")
    for i in range(len(tiles)):
        object_name = tiles["AbsVisImageFileName"].values[i]
        filename = object_name.split("/")[-1]
        os.makedirs(path_download, exist_ok=True)
        path_to_save = os.path.join(path_download, filename)

        if not os.path.exists(path_to_save):
            !aws s3 cp s3://$s3_bucket/$object_name $path_to_save

download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M163/M163_R43_C3_VIS.jpg to ../annotations/968129_1/M163_R43_C3_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M163/M163_R106_C2_VIS.jpg to ../annotations/968129_1/M163_R106_C2_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M163/M163_R105_C2_VIS.jpg to ../annotations/968129_1/M163_R105_C2_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M163/M163_R166_C2_VIS.jpg to ../annotations/968129_1/M163_R166_C2_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M163/M163_R165_C0_VIS.jpg to ../annotations/968129_1/M163_R165_C0_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M163/M163_R183_C0_VIS.jpg to ../annotations/968129_1/M163_R183_C0_VIS.jpg
download: s3://aqc-lambda-tiles-prod/cqt_qualitex_fino/specola_20/M163/M163_R141_C2_VIS.jpg to ../annotations/968129_1/M163_R141_C2_VIS.jpg
download: s3://aqc-lam

## Generating qualitex annotation per file 

In [12]:
text = ""
group_id = None
shape_type = "polygon"
flags = {}
height = 1024
width = 4096
path_to_save = "../annotations/"

for roll_id in filter_rolls:
    disposition_n = df_rolls[df_rolls["Id"] == roll_id]["DispositionNumber"].values[0]
    disposition_r = df_rolls[df_rolls["Id"] == roll_id]["DispositionRowNumber"].values[0]
    disposition = "_".join([str(disposition_n), str(disposition_r)])
    rolls = df_rolls[(df_rolls["DispositionNumber"] == disposition_n) &
                    (df_rolls["DispositionRowNumber"] == disposition_r)]
    tiles = df_tiles[(df_tiles["RollId"].isin(rolls.Id.values)) &
                    (df_tiles["Tags"].str.contains("\[]"))]
    defects = df_defects[(df_defects["TileId"].isin(tiles.Id.values)) &
                        (df_defects["CreatedBy"] == "operator")]
    tiles = tiles[tiles["Id"].isin(defects.TileId.values)]
    
    for i in range(len(tiles)):
        annotations = {"version": "0.3.3",
                    "flags": flags,
                    "shapes": []
                    }
        
        anno_name = tiles["AbsVisImageFileName"].values[i].split("/")[-1][:-4] + ".json"
        tile_id = tiles["Id"].values[i]
        tile_defects = defects[defects["TileId"] == tile_id].reset_index(drop=True)
        for j in range(len(tile_defects)):
            anno = {}
            x = json.loads(tile_defects["ShapePointsX"][j])[:-1]
            y = json.loads(tile_defects["ShapePointsY"][j])[:-1]
            label = tile_defects["Type"][j]
            points = []

            for k in zip(x,y):
                points.append(tuple([k[0]*width,k[1]*height]))

            anno["label"] = str(int(label))
            anno["text"] = text
            anno["points"] = points
            anno["group_id"] = group_id
            anno["shape_type"] = shape_type
            anno["flags"] = flags

            annotations["shapes"].append(anno)
            annotations["imagePath"] = tiles["AbsVisImageFileName"].values[i].split("/")[-1]
            annotations["imageData"] = None
            annotations["imageHeight"] = height
            annotations["imageWidth"] = width
        
        # Save annotations from qualitex in anylabeling format
        path_annotations = os.path.join(path_to_save, disposition, anno_name)
        if not os.path.exists(path_annotations):
            with open(path_annotations, "w") as j:
                json.dump(annotations, j)

        
        